## Tokenization

Lecture 2 | CMU ANLP Fall 2025 | Instructor: Sean Welleck

This is a notebook for [CMU CS11-711 Advanced NLP](https://cmu-l3.github.io/anlp-fall2025/) that explores tokenizers.

*Acknowledgements*: based on "[Let's build the GPT Tokenizer](https://www.youtube.com/watch?v=zduSFxRajkE)" by Andrej Karpathy, and its associated notebook.

### Ascii, Unicode, and UTF-8

We want to support tokenizing any string of text, which includes text in a variety of languages, code, Latex, etc.

In [1]:
"元気ですか。Hello!"

'元気ですか。Hello!'

Tokenization requires choosing a vocabulary, i.e. the set of possible tokens.

The set of [ASCII](https://en.wikipedia.org/wiki/ASCII) characters (e.g. a-zA-Z0-9) is not expressive: it cannot, for instance, represent a Japanese sentence.

A potential expressive vocabulary is the set of [Unicode](https://en.wikipedia.org/wiki/Unicode) characters:

In [2]:
[ord(x) for x in "元気ですか。Hello!"]

[20803, 27671, 12391, 12377, 12363, 12290, 72, 101, 108, 108, 111, 33]

However, there are around [150,000](https://en.wikipedia.org/wiki/Unicode) characters, and the Unicode standard may change. Hence we would have a large vocabulary, would need to support changes to the Unicode standard, and it would be inefficient. For example, the string above requires 12 tokens.

A third option is to observe that UTF-8 maps Unicode into byte strings. There are 256 such byte strings, i.e. a vocabulary of 256 tokens.

In [3]:
utf = "元気ですか。Hello!".encode("utf-8")

print([x for x in utf])

[229, 133, 131, 230, 176, 151, 227, 129, 167, 227, 129, 153, 227, 129, 139, 227, 128, 130, 72, 101, 108, 108, 111, 33]


This vocabulary is expressive: it can represent any unicode sequence. 

However, it is inefficient. For example, the string above now requires 24 tokens.

### Byte Pair Encoding

A middle ground is to start with the size-256 UTF-8 vocabulary, and merge elements of the vocabulary to add additional vocabulary elements. This is the approach taken in [GPT-2 (see section 2.2)](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf).

We will walk through one step of this merging procedure. Then, we will implement an outer loop. The resulting algorithm is called [Byte Pair Encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding).

First, here is an example text and its tokens.

In [4]:
text = """Hello, world! Here is some example text to test
the BPE algorithm. It is not very interesting, but it will
do the job.
""" 

tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

print(len(text))
print(len(tokens))

119
119


Next, we count the occurrences of each consecutive token pair (i.e., bigram).

Let's look at the top pair.

In [5]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts
    
stats = get_stats(tokens)
# print(sorted(((v,k) for k,v in stats.items()), reverse=True))

top_pair = max(stats, key=stats.get)
print(top_pair, '"' + chr(top_pair[0]) + '"', '"' + chr(top_pair[1]) + '"')

(101, 32) "e" " "


Now we want to **merge**. Specifically, we designate a new token. We replace all occurrences of the top pair with the new token.

In [6]:
def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

In [7]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print(len(tokens), len(tokens2))

[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33, 32, 72, 101, 114, 256, 105, 115, 32, 115, 111, 109, 256, 101, 120, 97, 109, 112, 108, 256, 116, 101, 120, 116, 32, 116, 111, 32, 116, 101, 115, 116, 10, 116, 104, 256, 66, 80, 69, 32, 97, 108, 103, 111, 114, 105, 116, 104, 109, 46, 32, 73, 116, 32, 105, 115, 32, 110, 111, 116, 32, 118, 101, 114, 121, 32, 105, 110, 116, 101, 114, 101, 115, 116, 105, 110, 103, 44, 32, 98, 117, 116, 32, 105, 116, 32, 119, 105, 108, 108, 10, 100, 111, 32, 116, 104, 256, 106, 111, 98, 46, 10]
119 114


Now let's put it into a loop, and run it on a longer sequence of text.

In [8]:
training_text = """Hello, world! 
Here is some example text to test
the BPE algorithm. It is not very 
interesting, but it will do the job.
""" 

# Use this one for a longer example. 
training_text = """大阪マラソン2025大会要項
大会名称	大阪マラソン2025　～OSAKA MARATHON 2025～（第13回大阪マラソン）
（英文名：Osaka Marathon 2025）
兼 ジャパンマラソンチャンピオンシップシリーズ・男子G1/女子G2
兼 東京2025世界陸上競技選手権大会 日本代表選手選考競技会（男子）
主催	大阪府、大阪市、（公財）大阪陸上競技協会
共催	読売新聞社、毎日新聞社、NHK、（公財）日本陸上競技連盟
主管	（公財）大阪陸上競技協会
後援	大阪市地域振興会、大阪府商店街連合会、大阪府商店街振興組合連合会、大阪市商店会総連盟、（公社）関西経済連合会、大阪商工会議所、（一社）関西経済同友会、（公財）大阪観光局、（公財）大阪府スポーツ協会、大阪府体育連合、大阪府スポーツ推進委員協議会、大阪市スポーツ協会、大阪市体育厚生協会、大阪市スポーツ推進委員協議会、（一財）大阪スポーツみどり財団、大阪府障がい者スポーツ協会、（社福）大阪市障害者福祉・スポーツ協会、（一社）大阪府医師会、（一社）大阪府病院協会、（公社）大阪府看護協会、国土交通省近畿地方整備局、国土交通省近畿運輸局、阪神高速道路（株）、（社福）読売光と愛の事業団、（特非）大阪ライフサポート協会、大阪府教育委員会、大阪市教育委員会、（株）報知新聞社、讀賣テレビ放送（株）、（株）毎日放送、（株）スポーツニッポン新聞社（予定、順不同）
テレビ放送	NHK、読売テレビ、毎日放送
種目	
マラソン(42.195㎞）
720<なにわ>マラソン(ランの部7.2㎞・車いすの部720m）
開催日時	
2025年（令和7年）2月24日（月・振替休日）

  9:15／
マラソン第1ウェーブスタート、以降第2ウェーブ・第3ウェーブを順次スタート
720<なにわ>マラソン（ランの部）第3ウェーブよりスタート
10:30／
720<なにわ>マラソン（車いすの部）スタート
11:00／
720<なにわ>マラソン（車いすの部）終了
11:05／
720<なにわ>マラソン（ランの部）終了
16:15／
マラソン終了
コース	
マラソン
大阪府庁前をスタートし、大阪城公園内をフィニッシュとする大阪マラソンコース（日本陸上競技連盟（日本陸連）・ワールドアスレチックス（WA）／国際マラソン・ディスタンスレース協会（AIMS）公認コース）
720<なにわ>マラソン
ランの部：大阪府庁前をスタートし、こども本の森中之島前をフィニッシュとするコース
車いすの部：城見１交差点をスタートし、大阪城公園内をフィニッシュとするコース
競技規則	最新のWA競技規則並びに日本陸連規則及び本大会規定によります。
なお、本大会はWA認定のラベルレースのため、WAロードレースラベリング規定が適用されます。また、WAの規則により、ドーピング検査を実施します。
スタート方法	
混雑緩和と選手安全対策のためウェーブ（時間差）スタートを実施します。日本陸連登録の有無に関わらず、申込時の記録証タイム（自己ベストタイム）の申告等を参考にして、ウェーブスタート順やスタート整列ブロックを設定します。記録証タイムと予想タイムの両方が未申告の場合は、最終ウェーブの最後尾ブロックからのスタートとします。
なお、設定されたウェーブよりも前方からスタートした場合は、失格とします。

【ウェーブスタート順と整列ブロック分けの優先順位】

招待選手・エリートランナー
 自己ベストタイム（グロスタイム又はネットタイム）をお持ちの方（2021年（令和3年）8月1日以降で確認できるマラソン大会での記録であること。虚偽の場合は出場を取り消します）
予想タイムを申告した方
自己ベストタイム・予想タイムの両方を申告しなかった方
グループエントリーの方は、申込時の自己ベストタイムと予想タイムが最も遅い方と同じブロックからのスタートとなります。
720<なにわ>マラソン（ランの部）は第3ウェーブからのスタートとなります。
制限時間	
マラソン：7時間（競技終了時刻16:15）
制限時間は第1ウェーブの号砲を基準とします。

720<なにわ>マラソン
ランの部：1時間20分（競技終了時刻11:05）
制限時間は第3ウェーブの号砲（9:45）を基準とします。

車いすの部：30分（競技終了時刻11:00）
仮装	日本陸連登録競技者は仮装を禁止します。Aブロックにおいては日本陸連の登録の有無に関わらず、仮装を禁止します。加えて、他のランナーや沿道の方に不快感を与える服装や行為は認めません。
参加資格	
マラソン：2006年（平成18年）4月1日以前に生まれた方
①日本陸連登録競技者（2024年度の登録者）
②日本陸連に登録していないランナー
①②ともに単独での走行が困難な方は伴走者（1名）をつけることができます。
（伴走者は、コース上の指定された箇所で１回に限り交代できます。なお、盲導犬等の動物の伴走は認めません。）
競技終了時刻までに完走できる方。
720<なにわ>マラソン
ランの部　：2009年（平成21年）4月1日以前に生まれた方
競技終了時刻までに完走できる方。
車いすの部：2012年（平成24年）4月1日以前に生まれた方。
身体障害者手帳を所持し車いすを使用される方。
使用できる車いすは、生活用車いす及び競技用車いす（陸上競技用のレーサーは除く）とします。
電動車いすは不可とします。
競技終了時刻までに完走できる方。
自力走行で競技終了時刻までに完走が困難とされる方は、介助する伴走者（1名）をつけることができます。
定員	34,000人（＜１＞マラソン：31,970人、＜２＞720〈なにわ〉マラソン（ランの部：2,000人、車いすの部：30人））
申込区分
（エリート部門を除く）	
マラソン
①一般ランナー（個人）国内・国外
②一般ランナー（グループ2人～7人）国内・国外
①～②合計28,420人

③障がい者ランナー（50人）国内
身体障害者手帳、精神障害者保健福祉手帳、療育手帳のいずれかをお持ちの方。
落選した場合は一般ランナーとして再抽選を行います（再エントリーは不要）。
④市民アスリート（1,500人/先着順）国内
年代・性別毎に設定した基準タイム以内の記録（日本陸連公認又はAIMS公認コースで2021年（令和3年）8月1日以降のグロスタイム又はネットタイム）を有する方に限ります。基準タイムは記録証に記載の大会当日満年齢によります。
18～39歳：3時間（男子）・3時間45分（女子）、
40～49歳：3時間10分（男子）・3時間50分（女子）、
50～59歳：3時間25分（男子）・4時間10分（女子）、
60～69歳：3時間50分（男子）・4時間40分（女子）、
70歳～　：4時間30分（男子）・5時間20分（女子）
資格審査に通らなかった場合は、事務手数料を差し引いて参加料等を返金します。一般ランナーとして参加を希望する場合は、2024年8月28日（水）17時までにご自身でエントリー手続をお願いします。
⑤大阪スポーツ応援ランナー（大阪府・大阪市各400人、計800人/先着順）国内
ふるさと納税制度を活用し、「大阪スポーツ応援ランナー」への寄附として、大阪府「なみはやスポーツ振興基金」又は大阪市「大阪市スポーツ振興基金」へ10万円以上寄附された方（もしくは寄附者が指名した方）に1名分の出走権を進呈します（参加料は別途必要）。
10万円以上の寄附を頂いた方に、エントリーコードをメールでお送りします。ランネットよりエントリーコードを入力の上、大会エントリー手続を完了してください（寄附申込だけでは、大阪マラソンへ参加ができませんので注意してください）。
⑥チャリティランナー（1,000人/先着順）国内・国外
参加料とは別に2口以上（1口500円）の寄附を行うとともにファンドレイジングにより、別途69,000円以上の寄附を集めていただきます。
ファンドレイジング期間内［2024年（令和6年）7月23日（火）10:00〜12月13日（金）17:00］に寄附金総額が最低寄附金額（7万円）に達しなかった場合は、不足金額を参加者ご自身に寄附いただきます。（クレジットカードへ不足額を自動的に課金します。最低寄附金額に到達しないことで、参加を取り消すことはできません）
⑦万博チケット付きランナー（200人/先着順）国内
大阪・関西万博2025 超早期購入割引チケット付きエントリーとなります。
720<なにわ>マラソン
①ランの部（2,000人）国内・国外
②車いすの部（30人）国内
申込方法	
国内
ランネット（https://runnet.jp/runtes/）を通じて、インターネットにて申込を受け付けます。
国外
JTBスポーツステーション（https://jtbsports.jp）を通じて、インターネットにて申込を受け付けます。
申込期間	
マラソン
①②一般ランナー③障がい者ランナー
2024年（令和6年）7月23日（火）10:00〜8月28日（水）17:00

④市民アスリート
2024年（令和6年）7月22日（月）10:00〜7月24日（水）17:00（先行募集・先着順）

⑤大阪スポーツ応援ランナー
寄附申込期間：2024年（令和6年）7月23日（火）10:00〜10月2日（水）17:00（先着順）
※但し、納付書による寄附は8月16日（金）17:00まで
エントリー期間：エントリーコード発行日〜10月16日（水）17:00

⑥チャリティランナー
2024年（令和6年）7月23日（火）10:00〜10月16日（水）17:00（先着順）
※ファンドレイジング期間は2024年（令和6年）7月23日（火）10:00〜12月13日（金）17:00

⑦万博チケット付きランナー
2024年（令和6年）7月23日（火）10:00〜8月28日（水）17:00（先着順）

720<なにわ>マラソン
①ランの部
2024年（令和6年）7月23日（火）10:00〜8月28日（水）17:00

②車いすの部
2024年（令和6年）9月1日（日）10:00～9月30日（月）17:00

参加者決定方法	
マラソン
①②一般ランナー、③障がい者ランナー
申込者数が定員を超えた場合は抽選を行い、抽選結果は、9月26日（木）にメールで通知します。（状況により追加当選通知を行う場合があります。）

当選通知の際に指定する期日までに支払手続を完了した時に、参加権利が確定します。

④市民アスリート、⑤大阪スポーツ応援ランナー、⑥チャリティランナー、⑦万博チケット付きランナー
先着順とし、定員になり次第締め切ります。

但し、④市民アスリートは資格審査結果に不備のある方のみ、申込締切後概ね3週間以内にメールで通知します。

720<なにわ>マラソン
①ランの部
申込者数が定員を超えた場合は抽選を行い、抽選結果は、9月26日（木）にメールで通知します。（状況により追加当選通知を行う場合があります）
②車いすの部
申込者数が定員を超えた場合は抽選を行い、抽選結果は、10月中にメールで通知します。（状況により追加当選通知を行う場合があります）

当選通知の際に指定する期日までに支払手続を完了した時に、参加権利が確定します。

参加料等	
マラソン
①一般ランナー（個人）国内：16,000円/国外：18,000円
②一般ランナー（グループ2～7人）参加者1名につき、国内：16,500円/国外：18,500円
③障がい者ランナー、④市民アスリート、⑤大阪スポーツ応援ランナー 国内：16,000円
⑥チャリティランナー　国内：16,000円　国外：18,000円
⑦万博チケット付きランナー　国内：22,000円
参加料とは別に事務手数料（国内：決済金額の5.5％、国外：決済金額の11%）及び参加者1人につき2口以上（1口500円）のチャリティ募金が必要です。
720<なにわ>マラソン
①ランの部　国内：5,000円/国外：6,000円

②車いすの部　国内：1,000円

参加料とは別に事務手数料（国内：決済金額の5.5％、国外：決済金額の11%※決済金額が4,000円以下の場合は一律220円）及び参加者1人につき2口以上（1口500円）のチャリティ募金が必要です。
参加料の支払方法	
クレジットカード等による即時決済またはコンビニ支払い。

(1)マラソンの④市民アスリートはクレジットカード等による即時決済のみ。

表彰	
表彰は、グロスタイムにより次のとおり行います。(720<なにわ>マラソンの表彰はありません）

◎総合男女の各1位～8位を表彰します。
◎市民ランナー賞として、招待選手、エリートランナーを除く男女各1位を表彰します。
◎シカゴマラソン賞として、招待選手、エリートランナー、連携大会の代表選手、第12回大会までの同賞受賞者を除く大阪府内在住者の男女各1位を表彰します。
◎プラハマラソン賞として、招待選手、エリートランナー、連携大会の代表選手を除く、大阪府内在住者の男女各2位を表彰します。ただし、男女各1位がシカゴマラソン賞に該当しない場合は男女各1位を表彰します。
別途ネットタイムにより、招待選手・エリートランナー・総合入賞者を除く年代別5歳刻みの男女各1位～3位と、最高齢で完走されたスーパーシニア賞に対して賞状を後日送付します。

ランナー受付
（大阪マラソンEXPO2025）	
日程／2025年（令和7年）2月22日（土）～2月23日（日・祝）

場所／インテックス大阪

時間／ランナー受付：2月22日（土）11:00〜19:00、2月23日（日・祝）10:00〜18:00
展示エリア：2月22日（土）11:00〜19:30（最終入場 19:00）、2月23日（日・祝）10:00〜18:30（最終入場18:00）

大会当日（2月24日(月・振替休日)）の受付は行いません。
受付時に、本人確認を行うため、ランナー本人以外の代理受付は認めません。
受付の際、顔写真付きの本人確認書類を持参してください。
障がい者ランナーは申込時に登録した手帳（身体障害者手帳、精神障害者保健福祉手帳、療育手帳のいずれか）を持参してください。
720＜なにわ＞マラソン（車いすの部）の受付は大会当日（9:00～9:40）に行います。
ドーピング
コントロール	WAアンチ・ドーピング規程もしくは日本アンチ・ドーピング規程に基づいてドーピング検査を行います。大会前又は後の検査においては、尿又は血液（あるいはその両方）の採取を行います。該当者は指示に従って検査を受けてください。
TUE申請	
禁止表国際基準で定められる禁止物質・禁止方法を病気の治療目的で使わざるを得ない競技者は、治療使用特例（TUE）の申請を行ってください。詳細については、下記ウェブサイトを確認してください。

▼日本陸連医事委員会
（https://www.jaaf.or.jp/about/resist/medical/）

▼日本アンチ・ドーピング機構
（https://www.playtruejapan.org/）
禁止物質・禁止方法についてTUEが付与されている場合には、その証明書（コピーで可）をドーピング検査の際に検査員へ提出してください。

参加注意事項	
主催者は競技中の事故は応急処置に限り対応します。主催者に重大な過失がある場合を除き、補償は加入した傷害保険（見舞金補償）の範囲内となります。
チャリティプログラムの趣旨に賛同できない方の参加はご遠慮ください。
参加資格を満たさない方は参加できません。申込者以外の参加は認めません。
氏名、参加資格、記録証等の虚偽や不正が判明した場合は失格とし、参加を認めません。また今後の大会の申込も認めません。
申込後の連絡は、登録されたメールアドレスへの配信で行います。参加者の機器等の不具合やメール設定の不備、メールアドレスの変更等により主催者からのメールを受信できなかった場合、主催者は一切の責任を負いません。
会場へは公共の交通機関等をご利用ください。720＜なにわ＞マラソン（車いすの部）は別途案内します。但し、交通機関の遅延等により参加できなかった場合、主催者は一切の責任を負いません。
体調管理を行った上で参加してください。
貴重品や手荷物の紛失や盗難、損傷などについて主催者は一切責任を負いません。
広告目的で大会会場（コース上を含む）において、企業名、商品名等を意味する図案や文字、商標等を掲出したり、身につけて表示することは認めません。
大会の映像、写真、記事、位置情報、参加者の氏名、年齢、居住地（都道府県名又は市区町村名）、記録等のテレビ、新聞、雑誌、インターネット、主催者が発行する印刷物等への掲載権と肖像権は主催者に帰属します。
本大会は、「AbbottWMM Wanda Age Group World Championships」の予選会となる「AbbottWMM Wanda Age Group World Rankings」の対象大会であることから、大会結果（氏名・年齢・性別・国籍・記録）をアボット・ワールドマラソンメジャーズを運営するアメリカ合衆国所在のWORLD MARATHON MAJORS LLCへ提供します。「AbbottWMM Wanda Age Group World Championships」出場の対象となったランナーにはその旨の通知がWORLD MARATHON MAJORS LLCから届きます。なお、通知を受け取るには、あらかじめご自身でWORLD MARATHON MAJORS LLCに登録する必要があります。詳細はアボット・ワールドマラソンメジャーズについてをご確認ください。
大会の写真等について、主催者が委託する者がランナー向けに販売することがあります。
上記のほか、大会に関する事項については主催者の指示に従ってください。
申込注意事項	
ご利用の機器等（OS、ブラウザソフト、回線を含む）によって申込みできないことがあります。機器等の不具合等による申込みの遅れについて、主催者は一切の責任を負いません。
複数の申込区分での重複申込が判明した場合はすべての申込を失格とし、参加を認めません。 同一人物による重複申込が判明した場合はすべての申込を失格とし、参加を認めません。
申込後の自己都合による申込内容の変更、キャンセル、参加料の返金はできません。ただし、エントリーサイトのマイページで変更できる変更については、この限りではありません。
申込状況、抽選結果等の問合せについては、一切応じられません。
主催者は参加料等の領収書の発行は行いません。クレジットカード会社発行の利用明細書又は請求書をご利用ください。コンビニ払いの場合は、お振込みの控えを領収書に代えさせていただきます。なお、ランネットからエントリーした場合は、ランネットのマイページより領収書の発行が可能です。
エリート部門への参加を希望する場合は、エリート部門でご応募ください。エリートランナーの募集は、別途行います。
開催可否・中止判断	地震、風水害などの災害、感染症拡大、警察・消防の対応が必要な事故等の発生により、安全な大会運営が困難と判断した場合には、大会を中止します。大会中止の場合の参加料等については、中止までに要した経費等を差し引いた上で返金の有無および金額を決定します。
個人情報について	
主催者は個人情報の保護法令を遵守し、大阪マラソン組織委員会のプライバシーポリシーに従って、参加者の個人情報を取り扱います。プライバシーポリシーについては、下記ウェブサイトを確認してください。

▼プライバシーポリシー
https://www.osaka-marathon.com/2025/policy/

主催者または大阪マラソンコールセンターから申込内容に関する確認連絡をさせていただくことがあります。
取材に承諾いただいた参加者に限り、主催者が、共催者及び関係メディア等に個人情報を提供させていただきます。
その他	感染症対策について、国、大阪府、（公財）日本スポーツ協会及び（公財）日本陸上競技連盟等から方針又はガイドラインが示された場合には、それらに沿って対策を行います。
Guide to the Osaka Marathon 2025
Race Name	Osaka Marathon 2025 (13th Osaka Marathon)
also serves as: Japan Marathon Championship Series Men's G1/Women's G2
World Athletics Championships Tokyo 2025 Marathon, Japan National Team qualifying trials (Men)
Race Organizers	Osaka Prefectural Government, Osaka City, Osaka Athletics (OA)
Co-organizers	The Yomiuri Shimbun, The Mainichi Newspapers, Japan Broadcasting Corporation (NHK), Japan Association of Athletics Federations (JAAF)
Managing Organization	Osaka Athletics (OA)
Operational Supporter	Osaka Para Athletics Association
Supporting Organizations	Osaka City Community Promotion Association, Osaka Prefecture Shopping District Association, Osaka Prefectural Federation of Shopping Center Promotion Associations, Osaka City Shopping Streets Association, Kansai Economic Federation, Osaka Chamber of Commerce and Industry, Kansai Association of Corporate Executives, Osaka Convention & Tourism Bureau, Osaka Prefectural Sport Association, Osaka Athletic Association, Osaka Sport Promotion Council, Osaka City Sport Association, Osaka City Sports and Welfare Association, Osaka City Sport Promotion Council, Osaka City Sports and Greenery Association, Osaka Para-Sports Association, Osaka City Welfare and Sports Association for Persons with Disabilities, Osaka Medical Association, Osaka Hospital Association, Osaka Nursing Association, Kinki Regional Development Bureau of the Ministry of Land, Infrastructure, Transport and Tourism, Kinki District Transport Bureau of the Ministry of Land, Infrastructure, Transport and Tourism, Hanshin Expressway Company Limited, Yomiuri Light and Humanity Association, Osaka Life Support Association, Osaka Prefectural Board of Education, Osaka City Board of Education, The Hochi Shimbun Inc., Yomiuri Telecasting Corporation, Mainichi Broadcasting System, Sports Nippon Newspapers (planned, in no particular order)
Official Sponsors	Osaka Metro Co., Ltd.; OPTAGE Inc.; Mizuno Corporation; Duskin Co., Ltd.; Daiwa House Industry Co., Ltd.; MUFG Bank, Ltd.; others
Telecast	NHK, Yomiuri Telecasting Corporation, Mainichi Broadcasting System
Event	
Marathon (42.195 km)
720 <Naniwa> Marathon (Runners: 7.2 km, Wheelchairs: 720 m)
Date & Time	
Monday, February 24, 2025
9:15: Marathon Wave 1 starts. This is followed by Wave 2 and Wave 3, which start sequentially.
720 <Naniwa> Marathon (Runners) starts with Wave 3.
10:30: 720 <Naniwa> Marathon (Wheelchairs) starts.
11:00: 720 <Naniwa> Marathon (Wheelchairs) finishes.
11:05: 720 <Naniwa> Marathon (Runners) finishes.
16:15: Marathon finishes.
Course	
Marathon:
Osaka Marathon course starting in front of the Osaka Prefectural Government Building and finishing within Osaka Castle (official course of the Japan Association of Athletics Federations (JAAF), World Athletics (WA), and the Association of International Marathons and Distance Races (AIMS))
720 <Naniwa> Marathon :
Runners: Course starting in front of the Osaka Prefectural Government Building and finishing in front of Nakanoshima Children’s Book Forest
Wheelchairs: Course starting at the Shiromi 1 Intersection and finishing inside Osaka Castle Park
Competition Rules	The race will be conducted in accordance with the latest rules and regulations of the WA, the JAAF, and this event.
Because the Osaka Marathon is a Label Road Race certified by the WA, the WA Road Race Label Regulations also apply. Doping control tests will be conducted in accordance with WA Rules. The wheelchair marathon follows the rules and regulations of the World Para Athletics (WPA) and this event.
Line-up at Start	
To relieve congestion and protect athletes’ safety, a wave (corral) start will be used. A wave-start schedule and waiting blocks will be set up according to the certified time on the record certificate (personal best time) submitted at the time of application, regardless of whether or not you have JAAF membership. Those who have not specified both a certified time on the record certificate and an estimated finish time will start from the rearmost block of the last wave.
Please note that starting from a point ahead of your set wave will result in disqualification.

[Priority order for wave starts and waiting blocks]

Invited athletes / Elite runners
Those who have a personal best time (gross time or net time) (The time must be recorded at a confirmable race held on or after August 1, 2021. If it is found to be a false report, the entry will be canceled.)
Those who reported an estimated finish time
Those who reported neither a personal best time nor an estimated finish time
* Group entries will start from the same block as the participant in the same group with the slowest personal best time and estimated time at application.
* 720 <Naniwa> Marathon (Runners) shall start from Wave 3.
Time Limit	
Marathon: 7 hours (The competition ends at 16:15.)
* Time limits are based on the timing of the starting gun for Wave 1.

720 <Naniwa> Marathon:
Runners: 1 hour and 20 minutes (The competition ends at 11:05.)
* Time limits are based on the timing of the starting gun (9:45) for Wave 3.

Wheelchairs: 30 minutes (The competition ends at 11:00.)
Costume-clad Participation	JAAF members are prohibited from participating in the race dressed in costume. Costume-clad runners are not allowed to start from Block A regardless of having JAAF membership or not. Costumes and behavior that cause discomfort to other runners or people along the route are prohibited.
Qualifications	
<1> Marathon: Persons born on or before April 1, 2006

JAAF members (FY2024 registered members)
Non-JAAF members
* In both (1) and (2), those who have difficulty running on their own may each be accompanied by one escort runner. (An escort runner can change only once at a designated point on the course. Running with a guide dog or any other animal is not permitted.)
* Those who are capable of completing the race within the race time limits.
<2> 720<Naniwa>Marathon（Runners）: Persons born on or before April 1, 2009

Maximum Number of Participants	34,000 (Marathon: 31,970 / 720 <Naniwa> Marathon (Runners): 2,000)
Application Category
(excluding elite runners)	
<1> Marathon:

①General runners (individuals), ②General runners (2- to 7-person groups)
① to ② total 28,420
③Charity Runners 1,000
<2> 720 <Naniwa> Marathon:

①Runners 2,000
How to Apply	
Domestic entries
Applications are accepted on the Internet through Runnet
(https://runnet.jp/runtes/).
Overseas entries
Applications are accepted on the Internet through JTB Sports Station
(https://jtbsports.jp).
Application Period	
<1> Marathon:

①②General runners: From 10:00 on July 23 (Tue) to 17:00 August 28（Tue）, 2024
③Charity Runner: From 10:00 on July 23 (Tue) to 17:00 October 16（Wed）, 2024
* First come, first served. Applications will no longer be accepted when capacity is reached.

<2> 720 <Naniwa> Marathon:

④Runners: From 10:00 on July 23 (Tue) to 17:00 August 28（Tue）, 2024
How Participants are Selected	①②④If the number of applicants exceeds the capacity, a drawing will be held. Notification of the drawing results will be provided on September 26, 2024 (Thursday) by email.(Depending on the situation, a notification may be sent to those who are additionally selected as participants.)
③First come, first served. Applications will no longer be accepted when capacity is reached
Entry Fee	
<1> Marathon:

①General runners (individuals) Domestic: 16,000 yen / Overseas: 18,000 yen
②General runners (2- to 7-person groups), per participant Domestic: 16,500 yen / Overseas: 18,500 yen
③Charity runners Domestic: 16,000 yen / Overseas: 18,000 yen
<2> 720 <Naniwa> Marathon:

④Runners Domestic: 5,000 yen / Overseas: 6,000 yen
* For both marathons, a separate administrative fee (5.5% of the amount of payment for domestic entries, and 11% for overseas entries) and at least two charity donations per participant (500 yen per donation) are required in addition to the entry fee.
Payment Method	
Payment must be made immediately by credit card, etc., or at a convenience store.

Awards	
The awards will be granted based on the gross time as follows. (No awards are issued for the 720 <Naniwa> Marathon.)

◎The top eight male and female finishers overall will be awarded.
◎The Civic Runner Award is granted to male and female runners finishing in first place, excluding invited athletes and elite runners.
◎The Chicago Marathon Award is granted to male and female runners who live in Osaka Prefecture and finish in first place, excluding invited athletes, elite runners, representative runners for affiliate marathons, and the Chicago Marathon Award winners up through the 12th Osaka Marathon.
◎The Prague Marathon Award is granted to male and female runners who live in Osaka Prefecture and finish in second place, excluding invited athletes, elite runners, and representative runners for affiliate marathons. However, if a male or female runner finishing first place is not eligible for the Chicago Marathon Award, they shall receive the Prague Marathon Award instead.
* A certificate will also be sent separately at a later date to the top three male and female runners per five-year age bracket, excluding invited athletes, elite runners, and overall prize winners, according to their net time. The oldest runner to complete the marathon will be sent the Super Senior Award at a later date.

Runners' Registration
(Osaka Marathon EXPO 2025)	
Period: Two days on February 22 (Sat) and 23 (Sun, holiday), 2025

Venue: INTEX Osaka

Time/Runner registration: February 22 (Sat) 11:00 to 19:00, February 23 (Sun, holiday) 10:00 to 18:00
Exhibition area: February 22 (Sat) 11:00 to 19:30 (last admission: 19:00), February 23 (Sun, holiday) 10:00 to 18:30 (last admission: 18:00)

* Registration will not be accepted on the day of the race (Monday, February 24). Please bring a photo identification document with you for registration.Your registration will not be accepted if you do not bring a photo identification document with you.
* In order to verify the identity of the runner at the time of registration, no substitutes other than the runner himself/herself will be allowed to register on behalf of the runner.
Doping Control	Doping control tests will be conducted in accordance with the latest WA Anti-Doping Rules, the World Anti-Doping Code, and the Japan Anti-Doping Code. For doping control tests carried out before or after the race, a sample of urine and/or blood will be collected. The relevant people should take the test as instructed.
TUE Application	
Athletes who have to use prohibited substances/methods included on the Prohibited List for the purpose of treating a disease must file an application for a Therapeutic Use Exemption (TUE). For details, please refer to the websites below.

▼JAAF Medical Committee
(https://www.jaaf.or.jp/about/resist/medical/)

▼Japan Anti-Doping Agency (JADA)
(https://www.playtruejapan.org/)

If a TUE has been issued for a prohibited substance/method, submit a certificate of proof (copy acceptable) to the inspector during doping control testing.

Notes for Participating in the Race	
The organizer will only provide first aid in the event of an accident during the race. Except in the case of gross negligence by the organizer, compensation will be within the range of the engaged accident insurance (solatium payment).
The event organizing body will not accept applications from those who disagree with the concept of charity programs.
Those who do not meet the qualifications cannot participate in the race. No one but the selected applicants can participate in the race.
If any falsification of your name/qualifications/record certificate or other dishonest act is identified, you will be disqualified and you will not be allowed to participate in the race. Furthermore, applications for future races will also not be allowed.
After application, notifications will be sent to the registered email address. The event organizing body will take no responsibility if you cannot receive email messages sent by the event organizing body due to a malfunction in your device, incorrect email settings, a change in your email address, or the like.
Please use public transportation to reach the venue. Please note that the event organizing body will take no responsibility if you cannot participate in the race due to transportation delays.
Please be sure to manage your health before participating in the race.
The event organizing body will take no responsibility for any loss of, theft of, or damage to your belongings, including valuables and baggage.
It is prohibited to post or wear any designs, letters, trademarks, or the like representing company names or product names, etc. for advertising purposes within the event site (including the race course).
Usage and portrait rights of the following items belong to the event organizing body when they are used for TV broadcasting, newspapers, magazines, the Internet, and printed materials published by the event organizing body: images, photographs, and articles covering the race; location information; and entrants’ names, ages, addresses (prefecture or city) and records.
This race is covered by the AbbottWMM Wanda Age Group World Rankings, which is a qualifier for the AbbottWMM Wanda Age Group World Championships. Therefore, race results (names, ages, genders, nationalities, and records) are provided to World Marathon Majors LLC, a U.S.-based company that manages the Abbott World Marathon Majors. Runners who qualify to participate in the AbbottWMM Wanda Age Group World Championships will receive a notification to that effect from World Marathon Majors LLC. To receive this notification, you yourself must first register with World Marathon Majors LLC. For more information, see the About Abbott World Marathon Majors.
Photographs of the race may be sold to the runners by a party entrusted by the event organizing body.
For matters relating to the race, even if they are not included in the aforementioned notes, please follow instructions given by the event organizing body.
Notes for Applications	
You may be unable to apply depending on the device you are using (including OS, browser software, and network configuration). The event organizing body will take no responsibility for any delay in applications due to equipment malfunctions or other problems.
You cannot apply for multiple application categories. If duplicate applications by the same person are found, all applications will be disqualified, and you will not be allowed to participate in the race.
You cannot change or cancel your application for personal reasons after application. However, this does not apply to changes that can be made on My Page.
We will not respond to any inquiries regarding application status, drawing results, and other matters.
No receipts will be issued for the entry fee and other charges. Please use your credit card statement or bill issued by a credit card company as your receipt. In the case of payment at a convenience store, use the copy of the payment slip as the receipt. Please note that those who enter the event through Runnet can obtain a receipt from My Page in Runnet.
If you wish to participate in the race as an elite runner, please apply in the category of “elite runners.” Applications for elite runners will be accepted separately.
Judgment on Whether the Race Can Be Held or Will Be Canceled	If the safe operation of the race is judged to be difficult due to an earthquake, storm, flood, or any other natural disaster; the spread of an infectious disease; accident requiring a police or fire department response; etc., the race will be canceled. The amount of refund of the entry fee, etc. in the case of race cancellation will be determined after deducting the expenses incurred before the cancellation decision was made.
Personal Information	
The event organizing body abides by laws and regulations related to the protection of personal information and handles the personal information of the participants in accordance with the privacy policy of the Osaka Marathon Organizing Committee. For the privacy policy, please refer to the website below.

▼Privacy Policy
https://www.osaka-marathon.com/2025/en/policy/

* The event organizing body or the Osaka Marathon Call Center may contact participants to confirm the information stated on their application forms.
* The event organizing body will not provide any personal information on participants to the co-organizers or related mass media without the participant's consent.
Other Information	In the event that the Japanese government, Osaka Prefecture, the Japan Sport Association, the Japan Association of Athletics Federations, or other applicable body issues policies or guidelines regarding countermeasures against infectious diseases, appropriate measures will be taken in accordance with those policies or guidelines.
"""
# https://www.osaka-marathon.com/2025


In [9]:
# UTF-8 vocabulary
tokens = training_text.encode("utf-8")
tokens = list(map(int, tokens))

# Character-level vocabulary
# tokens = list(training_text)
len(tokens)

39298

In [10]:
# Settings
starting_vocab_size = 256
vocab_size = 300
num_merges = vocab_size - starting_vocab_size
ids = list(tokens)

# Full BPE algorithm
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = starting_vocab_size + i
    print(f"pair: {pair} freq: {stats[pair]}")
    print(f"merging {pair} into a new token {idx}\n")
    ids = merge(ids, pair, idx)
    merges[pair] = idx


pair: (227, 129) freq: 1457
merging (227, 129) into a new token 256

pair: (227, 131) freq: 985
merging (227, 131) into a new token 257

pair: (227, 130) freq: 709
merging (227, 130) into a new token 258

pair: (101, 32) freq: 469
merging (101, 32) into a new token 259

pair: (239, 188) freq: 384
merging (239, 188) into a new token 260

pair: (227, 128) freq: 337
merging (227, 128) into a new token 261

pair: (116, 104) freq: 287
merging (116, 104) into a new token 262

pair: (111, 110) freq: 279
merging (111, 110) into a new token 263

pair: (116, 105) freq: 253
merging (116, 105) into a new token 264

pair: (97, 110) freq: 199
merging (97, 110) into a new token 265

pair: (111, 114) freq: 193
merging (111, 114) into a new token 266

pair: (101, 114) freq: 192
merging (101, 114) into a new token 267

pair: (261, 129) freq: 187
merging (261, 129) into a new token 268

pair: (115, 32) freq: 187
merging (115, 32) into a new token 269

pair: (116, 32) freq: 186
merging (116, 32) into a ne

We can see the merges occurring and the new tokens being added to the vocabulary.

We can interpret each merge as "compression". That is, two tokens are replaced with 1, hence reducing the length of the sequence. Let us look at how the length changed, and the compression ratio.

In [11]:
print("original tokens length:", len(tokens))
print("new tokens length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

original tokens length: 39298
new tokens length: 29324
compression ratio: 1.34X


### Tiktoken

Now let's look at some tools and tokenizers in practice.

OpenAI released a tokenization library called Tiktoken that can be used for **inference**: i.e., given a trained tokenizer, encode or decode sequences with it.

The library cannot be used for **training** a tokenizer, i.e. running BPE on a corpus.

In [12]:
# !pip install tiktoken
import tiktoken

enc = tiktoken.get_encoding("gpt2")
print(enc.encode("Hello, こんにちは"))

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("Hello, こんにちは"))

[15496, 11, 23294, 241, 22174, 28618, 2515, 94, 31676]
[9906, 11, 220, 90115]


### GPT-2 Tokenizer

OpenAI also released their tokenization inference code for GPT-2.

https://github.com/openai/gpt-2/blob/master/src/encoder.py

You can take a look; it has code for encoding and decoding given the trained GPT-2 tokenizer.

They also have some interesting hacks and implementation details. You can watch Andrej Karpathy's video for a fun exploration of these details.

### Sentencepiece

Another common library is Sentencepiece:

https://github.com/google/sentencepiece

It can do training and inference. However, by default it does BPE on Unicode characters, rather than UTF-8 byte strings. Hence, a tokenizer trained with the default configuration could have "out of vocabulary" tokens; i.e., the tokenizer is not expressive. 

To solve this, you can set an option `byte_fallback=True` in order for the tokenizer to encode an out of vocabulary token as a UTF-8 byte sequence.

Let us **train** a tokenizer. We put the mixed English/Japanese text from before into `toy_tokenizer_txt.txt`, and run Sentencepiece.

In [13]:
import sentencepiece as spm

with open("toy_tokenizer_txt.txt", "w", encoding="utf-8") as f:
    f.write(training_text)

In [ ]:
import os

# There are a lot of sentencepiece options.
# If you are considering training your own tokenizer for a serious project
# or production, make sure you are aware of the exact setting you are running.
# Here we just make sure that we have set the vocab size and byte_fallback.
# For fun, we also split digits to show the use of such a rule.
options = dict(
  # input spec
  input="toy_tokenizer_txt.txt",
  input_format="text",
  model_prefix="toy_tokenizer",
  model_type="bpe",
  vocab_size=2048,
  byte_fallback=True,
  split_digits=True,
  num_threads=os.cpu_count(),
)

spm.SentencePieceTrainer.train(**options)

In [15]:
sp = spm.SentencePieceProcessor()
sp.load('toy_tokenizer.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [16]:
ids = sp.encode("hello, こんにちは マラソ マラソン marathon")
print(ids)

print([sp.id_to_piece(idx) for idx in ids])

[1290, 295, 1370, 1387, 1364, 1560, 1485, 1398, 1811, 1404, 1364, 329, 1437, 582, 960]
['▁he', 'll', 'o', ',', '▁', 'こ', 'ん', 'に', 'ち', 'は', '▁', 'マラ', 'ソ', '▁マラソン', '▁marathon']


In [17]:
# What happens if you set `split_digits=False`?
ids = sp.encode("123 1 21.93 893 $123 9823+12/43.323")
print(ids)

print([sp.id_to_piece(idx) for idx in ids])

[1364, 1386, 1384, 1419, 1364, 1386, 1364, 1384, 1386, 1396, 1471, 1419, 1364, 1461, 1471, 1419, 1364, 39, 1386, 1384, 1419, 1364, 1471, 1461, 1384, 1419, 46, 1386, 1384, 1415, 1429, 1419, 1396, 1419, 1384, 1419]
['▁', '1', '2', '3', '▁', '1', '▁', '2', '1', '.', '9', '3', '▁', '8', '9', '3', '▁', '<0x24>', '1', '2', '3', '▁', '9', '8', '2', '3', '<0x2B>', '1', '2', '/', '4', '3', '.', '3', '2', '3']


### Recap

#### Tokenizers
A **tokenizer** encodes raw text into a sequence of discrete tokens, and vice-versa (decoding).

We saw the *BPE* algorithm for training a tokenizer. Training means that we iteratively find the most frequent token pairs and merge them into a new vocabulary element.

We can then encode and decode tokens with the trained tokenizer.

#### Practical considerations

1. **Dataset:** Since we train a tokenizer on a dataset, the composition of the dataset has a large impact on the vocabulary. Consider the following:
    - What if the tokenizer training dataset only had English text?
    - What if the only code in the tokenizer training dataset was Python code?

2. **Vocabulary size**: We need to choose a vocabulary size as a hyperparameter. As we **increase the vocab size**:
    - Shorter sequence length for a given string of text.
    - Size of token embedding table and language model head increase.
    - Fewer examples per token; each token may be undertrained.
    - Less computation per semantic unit

If we decrease the vocab size (e.g. to UTF-8 bytes in the limit), the sequence length increases.

3. **Odd phenomena** follow from using BPE tokenization. For example:
    - Some numeric tokens may be grouped (e.g. `123`) while others arent (e.g. `134` -> `1, 3, 4`). 
    - If we end a prompt in a space, e.g. `world. `, the following token may begin in a character, e.g. `world. Next` gets tokenized into `world. `, `Next`. However, the training set may have used ` Next`, so the `Next` token is out of distribution. 
    
You can check out Andrej's video for an excellent discussion of these and other phenomena.